In [3]:
import geopandas as gpd
import pandas as pd
import os
# Treat '' and np.inf as NaN
pd.options.mode.use_inf_as_na = True

In [4]:
path = '~/Code/LILI/data/' 
cols = ["inno_01","inno_02","inno_03","inno_04","inno_05","inno_06","inno_07","inno_08","inno_09","inno_10","inno_12","inno_13","inno_15","inno_16","inno_17","inno_18","inno_19","inno_20","inno_21","inno_22","inno_23","inno_24","inno_25","inno_26","inno_27","inno_29","inno_30","inno_31","inno_32","inno_33","inno_34","inno_39"]
imp_cols = ["twitter_id","user_id","user_location","longitude","latitude","place_box_1_long","place_box_1_lat","place_box_2_long","place_box_2_lat","country","country_code","place_full_name","place_id","place_name"] + cols

df_allgeo = pd.DataFrame()

In [6]:
dtypes={'twitter_id': 'int64',
        'user_id': 'int64',
        'user_location': 'string',
        'longitude': 'float',
        'latitude': 'float',
        'place_box_1_long': 'string',
        'place_box_1_lat': 'string',
        'place_box_2_long': 'string',
        'place_box_2_lat': 'string',
        'country': 'string',
        'country_code': 'string',
        'place_full_name': 'string',
        'place_id': 'string',
        'place_name': 'string',
        'place_type': 'string',
        'inno_01': 'boolean',
        'inno_02': 'boolean',
        'inno_03': 'boolean',
        'inno_04': 'boolean',
        'inno_05': 'boolean',
        'inno_06': 'boolean',
        'inno_07': 'boolean',
        'inno_08': 'boolean',
        'inno_09': 'boolean',
        'inno_10': 'boolean',
        'inno_12': 'boolean',
        'inno_13': 'boolean',
        'inno_15': 'boolean',
        'inno_16': 'boolean',
        'inno_17': 'boolean',
        'inno_18': 'boolean',
        'inno_19': 'boolean',
        'inno_20': 'boolean',
        'inno_21': 'boolean',
        'inno_22': 'boolean',
        'inno_23': 'boolean',
        'inno_24': 'boolean',
        'inno_25': 'boolean',
        'inno_26': 'boolean',
        'inno_27': 'boolean',
        'inno_29': 'boolean',
        'inno_30': 'boolean',
        'inno_31': 'boolean',
        'inno_32': 'boolean',
        'inno_33': 'boolean',
        'inno_34': 'boolean',
        'inno_39': 'boolean'}

In [26]:
def PandasImp(path, imp_cols=imp_cols, dtypes = dtypes):
    df = pd.read_csv(
        path, 
        sep = '\t', 
        lineterminator='\n', 
        usecols=imp_cols, 
        false_values =['', ' ', "''", "' '", "", " ", 'False', 'Fal', "['', '', '']" ], #There is some weired stuff in that Data! Turning it into false Values.
        dtype = dtypes)
    return df

In [13]:
#create subfolder list
subfolders = []
for root, dirs, files in os.walk('../data'):
   for name in files:
      if root in subfolders:
         pass
      else:
         subfolders.append(root) 
subfolders.sort()

In [14]:
# Stack all the bool to one named column
def BoolStack(df_s):
    df_s['inno_nr'] = df_s[cols].idxmax(1).astype('string')
    df_n = df_s.drop(cols, axis=1)
    df_n['inno_nr'] = df_n['inno_nr'].astype('string')
    return(df_n)

In [27]:
#loop trough everything. One CSV takes ~33sek per Month: 70min total (on Linus' Machine)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#ignore the warning, append still works and concat is not implemented in dask yet
df_s= BoolStack(PandasImp('empty.csv'))

for folder in subfolders:
    print(folder)
    for root, dirs, files in os.walk(folder):
        files.sort()
        for file in files:
            df = PandasImp(f'{folder}/{file}')
            
            df['innovation'] = df[cols].sum(axis=1)
            df_i = df[df.innovation != 0]
            df_i = df_i.drop('innovation', axis=1)

            df_i = BoolStack(df_i)

            if df_i.shape[1] == df_s.shape[1]:
                df_s = df_s.append(df_i)
            else: print(folder, "--> wrong column count")

../data/expressions/2012/01
../data/expressions/2012/02
../data/expressions/2012/03
../data/expressions/2012/04
../data/expressions/2012/05
../data/expressions/2012/06
../data/expressions/2012/07
../data/expressions/2012/08
../data/expressions/2012/09
../data/expressions/2012/10
../data/expressions/2012/11
../data/expressions/2012/12
../data/expressions/2013/01
../data/expressions/2013/02
../data/expressions/2013/03
../data/expressions/2013/04
../data/expressions/2013/05
../data/expressions/2013/06
../data/expressions/2013/07
../data/expressions/2013/08
../data/expressions/2013/09
../data/expressions/2013/10
../data/expressions/2013/11
../data/expressions/2013/12
../data/expressions/2014/01
../data/expressions/2014/02
../data/expressions/2014/03
../data/expressions/2014/04
../data/expressions/2014/05
../data/expressions/2014/06
../data/expressions/2014/07
../data/expressions/2014/08
../data/expressions/2014/09
../data/expressions/2014/10
../data/expressions/2014/11
../data/expressions/

In [31]:
df_s.tail(5)

,twitter_id,user_id,user_location,longitude,latitude,place_box_1_long,place_box_1_lat,place_box_2_long,place_box_2_lat,country,country_code,place_full_name,place_id,place_name,inno_nr
6490,153387731521773569,191216844,Paraguay,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02
37741,153525325689655297,30079977,Perú ツ,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02
24976,154204266041585664,298356610,bogota,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02
130875,155841483243663360,285206171,Ibagué,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02
205594,164957110965243904,141093464,"Santo Domingo, Rep. Dom.",NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02
175188,165275798348115970,60773078,somewhere,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02
172687,166019695911243776,375960107,en mi casa :P,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02
42133,166195265311744002,377150767,"Guatemala, Guatemala",NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02
49595,166566205354811392,250358444,"Stgo., Chile",NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,inno_02


In [34]:
df_s.to_parquet('geocoding_sample_0424_2.parquet')